In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
882,I do admit that my review is from a 2006 point...,negative
901,**** WARNING: here be spoilers **** Why do I w...,negative
328,Dad (78) and I (46) both had a good time watch...,positive
200,"""Indian burial ground"": If those three words a...",negative
937,"Great characters, great acting, great dialogue...",positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
882,admit review point view nearly year making mov...,negative
901,warning spoiler waste hastily fleeing year wat...,negative
328,dad good time watching flick today guy primari...,positive
200,indian burial ground three word appear anywher...,negative
937,great character great acting great dialogue in...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
positive    262
negative    238
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
882,admit review point view nearly year making mov...,0
901,warning spoiler waste hastily fleeing year wat...,0
328,dad good time watching flick today guy primari...,1
200,indian burial ground three word appear anywher...,0
937,great character great acting great dialogue in...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=150)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
import os
import certifi

import httpx

os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["MLFLOW_TRACKING_INSECURE_TLS"] = "true"  # disable SSL verify

os.environ["MLFLOW_TRACKING_REQUESTS_VERIFY"] = "false"

orig_init = httpx.Client.__init__
def patched_init(self, *args, **kwargs):
    kwargs["verify"] = False
    return orig_init(self, *args, **kwargs)

httpx.Client.__init__ = patched_init
## -----------------------------------------------------------------------------

In [16]:
import dagshub

## Set Remote Tracking URI, experiments will be hosted on Dagshub
mlflow.set_tracking_uri('https://dagshub.com/parth57/Text_Classification_MLOPS_Project.mlflow')
## dagshub.init() internally makes HTTPS calls, Those calls go through requests
dagshub.init(repo_owner='parth57', repo_name='Text_Classification_MLOPS_Project', mlflow = True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2026-01-01 18:29:27,074 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/parth57/Text_Classification_MLOPS_Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "parth57/Text_Classification_MLOPS_Project"

2026-01-01 18:29:27,080 - INFO - Initialized MLflow to track repo "parth57/Text_Classification_MLOPS_Project"


Repository parth57/Text_Classification_MLOPS_Project initialized!

2026-01-01 18:29:27,082 - INFO - Repository parth57/Text_Classification_MLOPS_Project initialized!
/opt/anaconda3/envs/text_class/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dagshub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Experiment: artifact_location='mlflow-artifacts:/0be17746274d44ab8a07aed36a771488', creation_time=1767271448336, experiment_id='0', last_update_time=1767271448336, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 150)
        mlflow.log_param("test_size", 0.3)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-01 18:29:38,520 - INFO - Starting MLflow run...
/opt/anaconda3/envs/text_class/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dagshub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
2026-01-01 18:29:38,849 - INFO - Logging preprocessing parameters...
/opt/anaconda3/envs/text_class/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dagshub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/text_class/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dagshub.com'. Adding certificate verification is strongly

🏃 View run abundant-mink-640 at: https://dagshub.com/parth57/Text_Classification_MLOPS_Project.mlflow/#/experiments/0/runs/2368f8ea20fe45eba164b2e556bab7f4
🧪 View experiment at: https://dagshub.com/parth57/Text_Classification_MLOPS_Project.mlflow/#/experiments/0


/opt/anaconda3/envs/text_class/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dagshub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
